In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class InteriorPointSolver:
    def __init__(self, func, grad, hess, ineq_constraints, eq_constraints_mat, eq_constraints_rhs, x0):
        self.func = func
        self.grad = grad
        self.hess = hess
        self.ineq_constraints = ineq_constraints
        self.eq_constraints_mat = eq_constraints_mat
        self.eq_constraints_rhs = eq_constraints_rhs
        self.x0 = x0

    def solve(self, t0=1, mu=10, tol=1e-8, max_iter=100):
        return self.interior_pt(t0, mu, tol, max_iter)

    def interior_pt(self, t0, mu, tol, max_iter):
        x = self.x0
        t = t0
        for k in range(max_iter):
            print(f"Outer iteration {k}, t = {t}, x = {x}")
            x = self.newton_solver(x, t, tol)
            print(f"  x_newton = {x}")
            if t >= 1e10:
                break
            t *= mu
        return x

    def objective_function(self, x):
        return self.func(x)

    def gradient(self, x):
        return self.grad(x)

    def hessian(self, x):
        return self.hess(x)

    def inequality_constraints(self, x):
        return np.array([g(x) for g in self.ineq_constraints])

    def equality_constraints(self, x):
        return self.eq_constraints_mat @ x - self.eq_constraints_rhs

    def barrier_function(self, x, t):
        ineq_vals = self.inequality_constraints(x)
        if np.any(ineq_vals >= 0):
            return np.inf
        return self.objective_function(x) - t * np.sum(np.log(-ineq_vals))

    def barrier_gradient(self, x, t):
        ineq_vals = self.inequality_constraints(x)
        grad = self.gradient(x)
        for i in range(len(self.ineq_constraints)):
            if ineq_vals[i] >= 0:
                grad += (1 / t) * self.ineq_constraints[i](x) * self.grad(x)  # Adjust gradient contribution
        return grad

    def newton_solver(self, x, t, tol):
        max_iter = 50
        for i in range(max_iter):
            print(f"    Newton iteration {i}, x = {x}")
            grad_phi = self.barrier_gradient(x, t)
            H_phi = self.hessian(x)

            # Construct KKT matrix and RHS
            A = self.eq_constraints_mat
            b = self.eq_constraints_rhs

            ineq_vals = self.inequality_constraints(x)
            KKT_matrix = np.zeros((x.size + A.shape[0], x.size + A.shape[0]))
            KKT_matrix[:x.size, :x.size] = H_phi
            KKT_matrix[:x.size, x.size:] = A.T
            KKT_matrix[x.size:, :x.size] = A
            KKT_rhs = np.concatenate([
                -grad_phi - A.T @ np.ones_like(b),
                A @ x - b
            ])

            try:
                delta = np.linalg.solve(KKT_matrix, KKT_rhs)
            except np.linalg.LinAlgError:
                print("    LinAlgError in solving KKT system, returning current x")
                return x  # Return current x if solving fails

            dx = delta[:x.size]
            x_new = x - dx  # Update x using the Newton direction
            print(f"    Newton iteration {i}, dx = {dx}, x_new = {x_new}")

            if np.linalg.norm(x_new - x) < tol:
                print(f"    Converged in Newton solver after {i} iterations")
                return x_new

            x = self.backtracking_line_search(x, dx, t)  # Perform line search to update x

        print("    Newton solver did not converge")
        return x

    def backtracking_line_search(self, x, dx, t, alpha=0.3, beta=0.7):
        step = 1.0
        c1 = 1e-4  # Parameter for Armijo condition (sufficient decrease)
        c2 = 0.9   # Parameter for curvature condition

        fx = self.barrier_function(x, t)
        grad_phi_x = self.barrier_gradient(x, t)
        grad_phi_dx = self.barrier_gradient(x + step * dx, t)

        iter_count = 0
        while iter_count < 50:  # Safety check to avoid infinite loop
            new_x = x + step * dx

            if np.all(self.inequality_constraints(new_x) < 0):  # Check if all constraints are satisfied
                new_fx = self.barrier_function(new_x, t)

                # Armijo condition (sufficient decrease)
                if new_fx <= fx + c1 * step * np.dot(grad_phi_x, dx):
                    # Curvature condition (Wolfe condition)
                    if np.dot(grad_phi_dx, dx) >= c2 * np.dot(grad_phi_x, dx):
                        return new_x

            # Reduce step size
            step *= beta
            iter_count += 1

        print("Warning: Max iterations reached in line search. Returning current x.")
        return x

    def plot_results(self, solution):
        x = np.linspace(-0.5, 1.5, 400)
        y = np.linspace(-0.5, 1.5, 400)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)

        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                z = 1 - X[i, j] - Y[i, j]
                if z >= 0:
                    Z[i, j] = self.objective_function([X[i, j], Y[i, j], z])
                else:
                    Z[i, j] = np.nan  # Outside feasible region

        plt.contour(X, Y, Z, levels=50, cmap='viridis')
        plt.colorbar()
        plt.plot(solution[0], solution[1], 'ro')  # Plot the solution point
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Objective Function Contours and Optimal Solution')
        plt.grid(True)
        plt.show()

# Example usage
def objective_function(x):
    return x[0]**2 + x[1]**2 + (x[2] + 1)**2

def gradient(x):
    return np.array([2*x[0], 2*x[1], 2*(x[2] + 1)])

def hessian(x):
    return np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2]
    ])

ineq_constraints = [
    lambda x: x[0],  # x >= 0
    lambda x: x[1],  # y >= 0
    lambda x: x[2]   # z >= 0
]

eq_constraints_mat = np.array([[1, 1, 1]])  # x + y + z = 1
eq_constraints_rhs = np.array([1])
x0 = np.array([0.1, 0.2, 0.7])  # Initial guess

solver = InteriorPointSolver(objective_function, gradient, hessian, ineq_constraints, eq_constraints_mat, eq_constraints_rhs, x0)
solution = solver.solve()
print("Optimal solution:", solution)

solver.plot_results(solution)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Objective function
def objective_function(x):
    return x[0]**2 + x[1]**2 + (x[2] + 1)**2

# Gradient of objective function
def gradient(x):
    return np.array([2*x[0], 2*x[1], 2*(x[2] + 1)])

# Inequality constraints (converted to suitable format for scipy.optimize.minimize)
def ineq_constraint_1(x):
    return x[0]

def ineq_constraint_2(x):
    return x[1]

def ineq_constraint_3(x):
    return x[2]

ineq_constraints = [
    {'type': 'ineq', 'fun': ineq_constraint_1},  # x >= 0
    {'type': 'ineq', 'fun': ineq_constraint_2},  # y >= 0
    {'type': 'ineq', 'fun': ineq_constraint_3}   # z >= 0
]

# Equality constraints
eq_constraints_mat = np.array([[1, 1, 1]])  # x + y + z = 1
eq_constraints_rhs = np.array([1])

# Initial guess
x0 = np.array([0.1, 0.2, 0.7])

# Consolidate constraints into a single list
all_constraints = [{'type': 'eq', 'fun': lambda x: eq_constraints_mat @ x - eq_constraints_rhs}] + ineq_constraints

# Perform constrained minimization
result = minimize(objective_function, x0, jac=gradient, constraints=all_constraints)

# Extract optimal solution
solution = result.x
print("Optimal solution:", solution)

# Plotting contours of the objective function
x = np.linspace(-0.5, 1.5, 400)
y = np.linspace(-0.5, 1.5, 400)
X, Y = np.meshgrid(x, y)
Z = objective_function([X, Y, 1 - X - Y])

plt.figure(figsize=(8, 6))
plt.contour(X, Y, Z, levels=50, cmap='viridis')
plt.colorbar()
plt.plot(solution[0], solution[1], 'ro')  # Plot the solution point
plt.xlabel('x')
plt.ylabel('y')
plt.title('Objective Function Contours and Optimal Solution')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class InteriorPointSolver:
    def __init__(self, func, grad, hess, ineq_constraints, eq_constraints_mat, eq_constraints_rhs, x0):
        self.func = func
        self.grad = grad
        self.hess = hess
        self.ineq_constraints = ineq_constraints
        self.eq_constraints_mat = eq_constraints_mat
        self.eq_constraints_rhs = eq_constraints_rhs
        self.x0 = x0

    def solve(self, t0=1, mu=10, tol=1e-8, max_iter=100):
        return self.interior_pt(t0, mu, tol, max_iter)

    def interior_pt(self, t0, mu, tol, max_iter):
        x = self.x0
        t = t0
        for k in range(max_iter):
            print(f"Outer iteration {k}, t = {t}, x = {x}")
            x = self.newton_solver(x, t, tol)
            print(f"  x_newton = {x}")
            if t >= 1e10:
                break
            t *= mu
        return x

    def objective_function(self, x):
        return self.func(x)

    def gradient(self, x):
        return self.grad(x)

    def hessian(self, x):
        return self.hess(x)

    def inequality_constraints(self, x):
        return np.array([g(x) for g in self.ineq_constraints])

    def equality_constraints(self, x):
        return self.eq_constraints_mat @ x - self.eq_constraints_rhs

    def barrier_function(self, x, t):
        ineq_vals = self.inequality_constraints(x)
        if np.any(ineq_vals >= 0):
            return np.inf
        return self.objective_function(x) - t * np.sum(np.log(-ineq_vals))

    def barrier_gradient(self, x, t):
        ineq_vals = self.inequality_constraints(x)
        grad = self.gradient(x)
        for i in range(len(self.ineq_constraints)):
            grad -= (t / ineq_vals[i]) * self.grad(x)
        return grad

    def newton_solver(self, x, t, tol):
        max_iter = 50
        for i in range(max_iter):
            print(f"    Newton iteration {i}, x = {x}")
            grad_phi = self.barrier_gradient(x, t)
            H_phi = self.hessian(x)

            # Construct KKT matrix and RHS
            A = self.eq_constraints_mat
            b = self.eq_constraints_rhs

            KKT_matrix = np.zeros((x.size + A.shape[0], x.size + A.shape[0]))
            KKT_matrix[:x.size, :x.size] = H_phi
            KKT_matrix[:x.size, x.size:] = A.T
            KKT_matrix[x.size:, :x.size] = A
            KKT_rhs = np.concatenate([
                -grad_phi,
                A @ x - b
            ])

            try:
                delta = np.linalg.solve(KKT_matrix, KKT_rhs)
            except np.linalg.LinAlgError:
                print("    LinAlgError in solving KKT system, returning current x")
                return x  # Return current x if solving fails

            dx = delta[:x.size]
            x_new = x + dx  # Update x using the Newton direction
            print(f"    Newton iteration {i}, dx = {dx}, x_new = {x_new}")

            if np.linalg.norm(dx) < tol:
                print(f"    Converged in Newton solver after {i} iterations")
                return x_new

            x = self.backtracking_line_search(x, dx, t)  # Perform line search to update x

        print("    Newton solver did not converge")
        return x

    def backtracking_line_search(self, x, dx, t, alpha=0.3, beta=0.7):
        step = 1.0
        fx = self.barrier_function(x, t)
        grad_phi_x = self.barrier_gradient(x, t)
    
        while True:  # Continue until a suitable step size is found
            new_x = x + step * dx
    
            if np.all(self.inequality_constraints(new_x) < 0):  # Check if all constraints are satisfied
                new_fx = self.barrier_function(new_x, t)
                
                # Recompute the gradient at new_x for the curvature condition check
                grad_phi_new_x = self.barrier_gradient(new_x, t)
    
                # Armijo condition (sufficient decrease)
                if new_fx <= fx + alpha * step * np.dot(grad_phi_x, dx):
                    # Curvature condition (Wolfe condition)
                    if np.dot(grad_phi_new_x, dx) >= beta * np.dot(grad_phi_x, dx):
                        return new_x
    
            # Reduce step size
            step *= beta
            print(f"      Step reduced to {step}")
    
            # Check for too small step size
            if step < 1e-10:
                print("Warning: Step size became too small. Returning current x.")
                return x


    def plot_results(self, solution):
        x = np.linspace(-0.5, 1.5, 400)
        y = np.linspace(-0.5, 1.5, 400)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)

        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                z = 1 - X[i, j] - Y[i, j]
                if z >= 0:
                    Z[i, j] = self.objective_function([X[i, j], Y[i, j], z])
                else:
                    Z[i, j] = np.nan  # Outside feasible region

        plt.contour(X, Y, Z, levels=50, cmap='viridis')
        plt.colorbar()
        plt.plot(solution[0], solution[1], 'ro')  # Plot the solution point
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Objective Function Contours and Optimal Solution')
        plt.grid(True)
        plt.show()

# Example usage
def objective_function(x):
    return x[0]**2 + x[1]**2 + (x[2] + 1)**2

def gradient(x):
    return np.array([2*x[0], 2*x[1], 2*(x[2] + 1)])

def hessian(x):
    return np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2]
    ])

ineq_constraints = [
    lambda x: x[0],  # x >= 0
    lambda x: x[1],  # y >= 0
    lambda x: x[2]   # z >= 0
]

eq_constraints_mat = np.array([[1, 1, 1]])  # x + y + z = 1
eq_constraints_rhs = np.array([1])
x0 = np.array([0.1, 0.2, 0.7])  # Initial guess

solver = InteriorPointSolver(objective_function, gradient, hessian, ineq_constraints, eq_constraints_mat, eq_constraints_rhs, x0)
solution = solver.solve()
print("Optimal solution:", solution)

solver.plot_results(solution)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class InteriorPointSolver:
    def __init__(self, func, grad, hess, ineq_constraints, eq_constraints_mat, eq_constraints_rhs, x0):
        self.func = func
        self.grad = grad
        self.hess = hess
        self.ineq_constraints = ineq_constraints
        self.eq_constraints_mat = eq_constraints_mat
        self.eq_constraints_rhs = eq_constraints_rhs
        self.x0 = x0

    def solve(self, t0=1, mu=10, tol=1e-8, max_iter=100):
        return self.interior_pt(t0, mu, tol, max_iter)

    def interior_pt(self, t0, mu, tol, max_iter):
        x = self.x0
        t = t0
        for k in range(max_iter):
            print(f"Outer iteration {k}, t = {t}, x = {x}")
            x = self.newton_solver(x, t, tol)
            print(f"  x_newton = {x}")
            if t >= 1e10:
                break
            t *= mu
        return x

    def objective_function(self, x):
        return self.func(x)

    def gradient(self, x):
        return self.grad(x)

    def hessian(self, x):
        return self.hess(x)

    def inequality_constraints(self, x):
        return np.array([x[0], x[1], x[2]])  # x >= 0, y >= 0, z >= 0

    def equality_constraints(self, x):
        return np.dot(self.eq_constraints_mat, x) - self.eq_constraints_rhs

    def barrier_function(self, x, t):
        ineq_vals = self.inequality_constraints(x)
        if np.any(ineq_vals < 0):
            return np.inf
        return self.objective_function(x) - t * np.sum(np.log(ineq_vals + 1e-6))

    def barrier_gradient(self, x, t):
        ineq_vals = self.inequality_constraints(x)
        grad = self.gradient(x)
        for i in range(len(ineq_vals)):
            if ineq_vals[i] < 0:
                grad -= (t / (ineq_vals[i] + 1e-6)) * self.grad(x)
        return grad

    def newton_solver(self, x, t, tol):
        max_iter = 100
        for i in range(max_iter):
            print(f"    Newton iteration {i}, x = {x}")
            grad_phi = self.barrier_gradient(x, t)
            H_phi = self.hessian(x)

            # Construct KKT matrix and RHS
            A = self.eq_constraints_mat
            b = self.eq_constraints_rhs

            KKT_matrix = np.zeros((x.size + A.shape[0], x.size + A.shape[0]))
            KKT_matrix[:x.size, :x.size] = H_phi
            KKT_matrix[:x.size, x.size:] = A.T
            KKT_matrix[x.size:, :x.size] = A
            KKT_rhs = np.concatenate([
                -grad_phi,
                A @ x - b
            ])

            try:
                delta = np.linalg.solve(KKT_matrix, KKT_rhs)
            except np.linalg.LinAlgError:
                print("    LinAlgError in solving KKT system, returning current x")
                return x  # Return current x if solving fails

            dx = delta[:x.size]
            x_new = x + dx  # Update x using the Newton direction
            print(f"    Newton iteration {i}, dx = {dx}, x_new = {x_new}")

            if np.linalg.norm(dx) < tol:
                print(f"    Converged in Newton solver after {i} iterations")
                return x_new

            x = self.backtracking_line_search(x, dx, t)  # Perform line search to update x

        print("    Newton solver did not converge")
        return x


    def backtracking_line_search(self, x, dx, t, alpha=0.01, beta=0.5, rho=0.5):
        step = 1.0
        fx = self.barrier_function(x, t)
        grad_phi_x = self.barrier_gradient(x, t)

        while step > 1e-10:  # Ensure step size doesn't become too small
            new_x = x + step * dx

            if np.all(self.inequality_constraints(new_x) >= 0):  # Check if all constraints are satisfied
                new_fx = self.barrier_function(new_x, t)
                grad_phi_new_x = self.barrier_gradient(new_x, t)

                # Armijo condition (sufficient decrease)
                if new_fx <= fx + alpha * step * np.dot(grad_phi_x, dx):
                    # Curvature condition (Wolfe condition)
                    if np.dot(grad_phi_new_x, dx) >= beta * np.dot(grad_phi_x, dx):
                        return new_x

            # Reduce step size
            step *= rho
            print(f"      Step reduced to {step}")

        print("Warning: Step size became too small. Returning current x.")
        return x





    def plot_results(self, solution):
        x = np.linspace(-0.5, 1.5, 400)
        y = np.linspace(-0.5, 1.5, 400)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)

        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                z = 1 - X[i, j] - Y[i, j]
                if z >= 0:
                    Z[i, j] = self.objective_function([X[i, j], Y[i, j], z])
                else:
                    Z[i, j] = np.nan  # Outside feasible region

        plt.contour(X, Y, Z, levels=50, cmap='viridis')
        plt.colorbar()
        plt.plot(solution[0], solution[1], 'ro')  # Plot the solution point
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Objective Function Contours and Optimal Solution')
        plt.grid(True)
        plt.show()

# Example usage
def objective_function(x):
    return x[0]**2 + x[1]**2 + (x[2] + 1)**2

def gradient(x):
    return np.array([2*x[0], 2*x[1], 2*(x[2] + 1)])

def hessian(x):
    return np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2]
    ])

ineq_constraints = [
    lambda x: x[0],  # x >= 0
    lambda x: x[1],  # y >= 0
    lambda x: x[2]   # z >= 0
]

eq_constraints_mat = np.array([[1, 1, 1]])  # x + y + z = 1
eq_constraints_rhs = np.array([1])
x0 = np.array([0.1, 0.2, 0.7])  # Initial guess

solver = InteriorPointSolver(objective_function, gradient, hessian, ineq_constraints, eq_constraints_mat, eq_constraints_rhs, x0)
solution = solver.solve()
print("Optimal solution:", solution)

solver.plot_results(solution)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def objective_function(x):
    return x[0]**2 + x[1]**2 + (x[2] + 1)**2

def gradient_function(x):
    return np.array([2*x[0], 2*x[1], 2*(x[2] + 1)])

def hessian_function(x):
    return np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2]
    ])

class InteriorPointMethod:
    def __init__(self, objective_function, gradient_function, hessian_function):
        self.objective_function = objective_function
        self.gradient_function = gradient_function
        self.hessian_function = hessian_function

    def minimize(self, x0, max_iterations=100):
        path = []
        x = x0.copy()
        path.append(x)

        t = 1.0  # Initial value for the barrier parameter
        mu = 10  # Factor to increase barrier parameter in each iteration

        for _ in range(max_iterations):
            # Define the log-barrier function phi
            phi = lambda x: self.objective_function(x) + np.sum(-np.log(x))

            # Gradient of the barrier function
            grad_phi = lambda x: self.gradient_function(x) - np.reciprocal(x)

            # Hessian of the barrier function
            hess_phi = lambda x: self.hessian_function(x) + np.diag(np.reciprocal(x**2))

            # Newton's method with backtracking line search for the barrier problem
            alpha = 0.5
            beta = 0.01
            d = np.linalg.solve(hess_phi(x), -grad_phi(x))
            t_new = t
            while np.any(x + t_new * d <= 0):
                t_new *= alpha
            while phi(x + t_new * d) > phi(x) + beta * t_new * np.dot(grad_phi(x), d):
                t_new *= alpha

            x = x + t_new * d
            path.append(x)

            t *= mu  # Increase the barrier parameter for the next iteration

        return np.array(path)

def plot_contour_with_path(objective_function, path):
    x = np.linspace(-2, 2, 400)
    y = np.linspace(-2, 2, 400)
    X, Y = np.meshgrid(x, y)
    Z = objective_function([X, Y, -1])

    plt.figure(figsize=(10, 8))
    plt.contour(X, Y, Z, levels=np.logspace(-1, 3, 10))
    plt.colorbar(label='Objective Function')
    plt.title('Objective Function Contour Plot with Optimization Path')
    plt.xlabel('x')
    plt.ylabel('y')

    path = np.array(path)
    plt.plot(path[:, 0], path[:, 1], marker='o', color='red', label='Optimization Path')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    starting_point = np.array([0.1, 0.2, 0.7])

    optimizer = InteriorPointMethod(objective_function, gradient_function, hessian_function)
    path = optimizer.minimize(starting_point)
    final_point = path[-1]
    print("Final Point:", final_point)

    plot_contour_with_path(objective_function, path)


In [1]:
def plot(self) -> None:
        '''
        Plot contour lines of the objective function and overlay the path taken by the optimizer.
        '''
        # Define a grid of points for plotting the contour lines
        x_vals = np.linspace(-5, 5, 100)
        y_vals = np.linspace(-5, 5, 100)
        X, Y = np.meshgrid(x_vals, y_vals)
        Z = np.zeros_like(X)

        # Evaluate the objective function at each point on the grid
        for i in range(len(x_vals)):
            for j in range(len(y_vals)):
                point = np.array([X[i, j], Y[i, j]])
                Z[i, j] = self.f(point)  # Check here for proper handling of input to self.f

        # Create contour plot for the objective function
        contour = go.Contour(x=x_vals, y=y_vals, z=Z, colorscale='Viridis', showscale=False)

        # Extract x and y coordinates from self.path
        x_path = self.path[:, 0]
        y_path = self.path[:, 1]

        # Create scatter plot for the optimizer path
        path_trace = go.Scatter(x=x_path, y=y_path, mode='lines+markers', name='Optimizer Path')

        # Layout settings
        layout = go.Layout(title='Optimizer Path and Contour Lines',
                           xaxis=dict(title='X'),
                           yaxis=dict(title='Y'))

        # Create figure and plot
        fig = go.Figure(data=[contour, path_trace], layout=layout)
        fig.update_layout(title_text="Optimizer Path and Contour Lines", title_x=0.5)

        # Display the plot
        fig.show()

    def plot_path_plotly(self):
        x = self.path[:, 0]
        y = self.path[:, 1]

        if self.path.shape[1] == 3:
            z = self.path[:, 2]

            # Create grids for x, y, and z
            x_grid, y_grid, z_grid = np.meshgrid(np.linspace(np.min(x), np.max(x), 50),
                                                 np.linspace(np.min(y), np.max(y), 50),
                                                 np.linspace(np.min(z), np.max(z), 50))

            # Compute function values f(x, y, z) on the grid
            z_function = np.array([[self.f_call(self.f, np.array([x_i, y_i, z_i]))[0]
                                    for x_i, y_i, z_i in zip(x_row, y_row, z_row)]
                                   for x_row, y_row, z_row in zip(x_grid, y_grid, z_grid)])

            # Create 3D surface plot with colored surface
            fig = go.Figure(data=[
                go.Surface(x=x_grid, y=y_grid, z=z_grid, surfacecolor=z_function, colorscale='Viridis')
            ])

            fig.update_layout(title='3D Path Plot with Colored Surface', autosize=False,
                              scene=dict(
                                  xaxis=dict(title='X'),
                                  yaxis=dict(title='Y'),
                                  zaxis=dict(title='Z'),
                                  aspectmode='cube',
                                  bgcolor='rgb(17,17,17)'  # Dark mode background color
                              ),
                              margin=dict(l=20, r=20, b=20, t=40),  # Adjust margins as needed
                              width=1920,
                              height=1080
                              )

            # Add the path as a scatter plot
            fig.add_trace(go.Scatter3d(
                x=x, y=y, z=z,
                mode='lines+markers',
                marker=dict(size=5, color='lightblue'),
                line=dict(color='royalblue', width=3),  # Thicker line for the path
                name='Path',
            ))

            # Highlight the last point in red with larger marker
            last_x, last_y, last_z = x[-1], y[-1], z[-1]
            fig.add_trace(go.Scatter3d(
                x=[last_x], y=[last_y], z=[last_z],
                mode='markers',
                marker=dict(size=8, color='red'),
                name='Last Point',
            ))

            fig.show()

        else:
            # Compute z values using the objective function on a grid
            x_grid, y_grid = np.meshgrid(np.linspace(np.min(x), np.max(x), 50),
                                         np.linspace(np.min(y), np.max(y), 50))
            z_function = np.array([[self.f_call(self.f, np.array([x_i, y_i, 0]))[0] for x_i in x_grid[0]] for y_i in y_grid[:, 0]])

            # Create 3D surface plot with uniform color
            fig = go.Figure(data=[
                go.Surface(x=x_grid, y=y_grid, z=z_function, colorscale='Viridis')
            ])

            fig.update_layout(
                scene=dict(
                    xaxis=dict(title='X Axis'),
                    yaxis=dict(title='Y Axis'),
                    zaxis=dict(title='Z Axis'),
                    aspectmode='cube',
                    bgcolor='rgb(17,17,17)'  # Dark mode background color
                ),
                title='3D Path Plot with Colored Surface',
                margin=dict(l=20, r=20, b=20, t=40),  # Adjust margins as needed
                width=1920,
                height=1080
            )

            # Add the path as a scatter plot
            z = []
            for point in self.path:
                z.append(self.f_call(self.f,point)[0])
            fig.add_trace(go.Scatter3d(
                x=x, y=y, z=z,
                mode='lines+markers',
                marker=dict(size=5, color='lightblue'),
                line=dict(color='royalblue', width=3),  # Thicker line for the path
                name='Path',
            ))

            # Highlight the last point in red with larger marker
            last_x, last_y = x[-1], y[-1]
            fig.add_trace(go.Scatter3d(
                x=[last_x], y=[last_y], z=[z[-1]],
                mode='markers',
                marker=dict(size=8, color='red'),
                name='Last Point',
            ))

            fig.show()

        # Display the plot
        

    # def plot_path_plotly(self) -> None:
    #     import plotly.graph_objects as go
    #     import numpy as np

    #     # Get x and y values from the path
    #     x = self.path[:, 0]
    #     y = self.path[:, 1]

    #     # Check if the objective function is 2-variable or 3-variable
    #     is_2d_function = len(self.path[0]) == 2

    #     if is_2d_function:
    #         # Compute z values using the objective function
    #         

    #         # Create figure for 2D path plot with uniform color surface
    #         fig = go.Figure(data=[
    #             go.Scatter3d(
    #                 x=x,
    #                 y=y,
    #                 z=z_values,
    #                 mode='lines+markers',
    #                 marker=dict(size=5, color='lightblue'),
    #                 line=dict(color='royalblue', width=2),
    #                 name='Path'
    #             ),
    #             go.Surface(
    #                 x=[x],
    #                 y=[y],
    #                 z=[z_values],
    #                 colorscale=[[0, 'rgba(255, 255, 255, 0.7)'], [1, 'rgba(255, 255, 255, 0.7)']],  # Uniform color
    #                 showscale=False,
    #                 name='Surface'
    #             )
    #         ])

    #         # Set layout properties
    #         fig.update_layout(
    #             scene=dict(
    #                 xaxis=dict(title='X Axis'),
    #                 yaxis=dict(title='Y Axis'),
    #                 zaxis=dict(title='Z Axis'),
    #                 aspectmode='cube'
    #             ),
    #             title='2D Path Plot with Uniform Color Surface',
    #             margin=dict(l=0, r=0, b=0, t=20),
    #             template='plotly_dark',  # Dark mode template
    #             width=1280,
    #             height=720
    #         )

    #     else:
    #         # Compute z values using the objective function on a grid
    #         x_grid, y_grid = np.meshgrid(np.linspace(np.min(x), np.max(x), 50),
    #                                      np.linspace(np.min(y), np.max(y), 50))
    #         z_function = np.array([[self.f_call(self.f, np.array([x_i, y_i, 0]))[0] for x_i in x_grid[0]] for y_i in y_grid[:, 0]])

    #         # Create figure for 3D path plot with colored surface
    #         fig = go.Figure(data=[
    #             go.Scatter3d(
    #                 x=x,
    #                 y=y,
    #                 z=z_values,
    #                 mode='lines+markers',
    #                 marker=dict(size=5, color='lightblue'),
    #                 line=dict(color='royalblue', width=2),
    #                 name='Path'
    #             ),
    #             go.Surface(
    #                 x=x_grid,
    #                 y=y_grid,
    #                 z=z_function,
    #                 colorscale='Viridis',  # Color based on z values
    #                 showscale=True,
    #                 name='Surface'
    #             )
    #         ])

    #         # Set layout properties
    #         fig.update_layout(
    #             scene=dict(
    #                 xaxis=dict(title='X Axis'),
    #                 yaxis=dict(title='Y Axis'),
    #                 zaxis=dict(title='Z Axis'),
    #                 aspectmode='cube'
    #             ),
    #             title='3D Path Plot with Colored Surface',
    #             margin=dict(l=0, r=0, b=0, t=20),
    #             template='plotly_dark',  # Dark mode template
    #             width=1280,
    #             height=720
    #         )

    #     # Show the plot
    #     fig.show()


Convergence achieved at iteration: 5, Function value: -2.1597557552302424, At: [1.46301532 0.69674044]
Convergence achieved at iteration: 5, Function value: -2.8193261240772007, At: [1.90521016 0.91411596]
Convergence achieved at iteration: 6, Function value: -2.9801994802407745, At: [1.99005025 0.99014923]
Convergence achieved at iteration: 5, Function value: -2.9980019994980025, At: [1.9990005 0.9990015]
Convergence achieved at iteration: 5, Function value: -2.9998000199995, At: [1.99990001 0.99990001]
Convergence achieved at iteration: 5, Function value: -2.9999800001999994, At: [1.99999 0.99999]
Convergence achieved at iteration: 5, Function value: -2.999998000002317, At: [1.999999 0.999999]
Convergence achieved at iteration: 5, Function value: -2.99999980000002, At: [1.9999999 0.9999999]
Convergence achieved at iteration: 5, Function value: -2.99999998, At: [1.99999999 0.99999999]
Convergence achieved at iteration: 5, Function value: -2.999999998, At: [2. 1.]
2
Figure({
    'data'

/Users/baselmassarweh/Desktop/NumericalOptimization/numerical_optimization_hw/programming_assignment2/src/constrained_min.py:292: RuntimeWarning: invalid value encountered in log
  objective -= np.log(-objective_i)
